## Irrigation strategies based on optimized soil-moisture thresholds


In [1]:
# !pip install aquacrop

## Imports

In [1]:
from aquacrop.classes import *
from aquacrop.core import *
from aquacrop.lars import *

from scipy.optimize import differential_evolution
import numpy as np
from functools import partial
import pandas as pd

from aquacropgym.envs import CropEnv, nebraska_maize_config
from aquacropgym.utils import evaluate_agent
from aquacropgym.utils import calc_eto_faopm


## Define Non-DRL agent that irrigates based on fixed set of soil-moisture thresholds

In [2]:
class FixedThresholdAgent():
    def __init__(self,smts):
        self.smts=smts # gievn as fraction

    
    def compute_single_action(self,obs,explore=False):
        """
        pass soil-moisture thresholds to env
        expects smts between -1, +1 so need to scale
        """
        return (self.smts*2)-1

## Champion - Maize example

### bounds for optimization

In [3]:
max_bound = np.ones(4)
min_bound = np.zeros(4)
bounds = [(min_bound[i], max_bound[i]) for i in range(4)]


### import weather data

In [4]:
gendf=calc_eto_faopm('data/CPWG.dat',1995,40.4,1072,True,["simyear","jday","minTemp","maxTemp","precip","rad"])

### crop simulation config options

In [5]:
test_env_config=nebraska_maize_config.copy() # get default config dictionary
test_env_config['gendf']=gendf # set weather data
test_env_config['normalize_obs']=False # normalize input observation (with a pre calculated mean and standard deviation)
test_env_config['include_rain']=True # include rainfall within weather data
test_env_config['observation_set']='default' # set of variables to pass to agent
test_env_config['max_irr']=25 # max irrigation that can be applied in a single irrigation event
test_env_config['action_set']='smt4' # action set
test_env_config['days_to_irr']=7 # does not matter for fixed dmt agent
test_env_config['max_irr_season']=10_000 # max amount of irrigation (mm/ha) that can be applied in a single season
test_env_config['evaluation_run']=True


### test evaluation function

returns profit over train and test years

In [6]:
evaluate_agent(FixedThresholdAgent(np.array([0.7]*4)),CropEnv,test_env_config)

(282.3254644178013, 288.4700266576486)

### fitness function to optimize. evaluates SMT strategy and return train profits to maximise

In [8]:
def optim_func(smt):
    train,test = evaluate_agent(FixedThresholdAgent(smt.reshape(-1)),CropEnv,test_env_config)
    
    return -train

### optimize smts

With access to more compute you will want to increase workers, popsize, maxiter, or decrease tol.

In [9]:
max_bound = np.ones(4)
min_bound = np.zeros(4)
bounds = [(min_bound[i], max_bound[i]) for i in range(4)]

res = differential_evolution(optim_func,bounds=bounds,disp=True,workers=1,seed=42,tol=0.1,popsize=5,maxiter=2)

differential_evolution step 1: f(x)= -465.514
differential_evolution step 2: f(x)= -501.72


In [11]:
res

     fun: -501.72038719127033
 message: 'Maximum number of iterations has been exceeded.'
    nfev: 65
     nit: 2
 success: False
       x: array([0.69242557, 0.27945337, 0.35216182, 0.13150515])

### Get train and test year performance of optimized smts

In [12]:
evaluate_agent(FixedThresholdAgent(res.x.reshape(-1)),CropEnv,test_env_config)

(501.72038719127033, 517.3620008682144)

### More thourough optimization

By increasing optimization params to `workers=8,tol=0.01,popsize=5,maxiter=1000`
as done for the article, the optimal smts were found to be `[0.69892958, 0.56825608, 0.35286359, 0.11124398]`

In [13]:
evaluate_agent(FixedThresholdAgent(np.array([0.69892958, 0.56825608, 0.35286359, 0.11124398])),CropEnv,test_env_config)

(513.0584847087756, 521.3771423907657)